In [1]:
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import config
import utility.df
import utility.iolib
import utility.plotlib
from utility.feature import Feature

#env = sys.argv[1] if len(sys.argv) > 2 else "dev"

In [6]:
# Setup configuration
cfg = config.ResearchConfig
time_format = cfg.CSV_TIME_FORMAT
date_format = cfg.CSV_DATE_FORMAT
cutoff_date = pd.to_datetime(cfg.CUTOFF_DATE, format=cfg.CSV_DATE_FORMAT)

# Retrieve data
df_subspt, df_lesson, df_incomp, df_crclum, df_pupils = utility.iolib.retrieve_data(cfg)
print("Complete loading data for subscription and lesson history!")

# Filter data
cutoff_date = pd.to_datetime(cfg.CUTOFF_DATE, format=cfg.CSV_DATE_FORMAT)
first_date_impFromData = df_subspt.subscription_start_date.min()

pupils_toBeRemoved = utility.df.filter_subspt_data(
    df_subspt, first_date_impFromData, cutoff_date, remove_annual_subspt=False)
df_lesson1 = df_lesson[~df_lesson['pupilId'].isin(pupils_toBeRemoved)]
df_incomp1 = df_incomp[~df_incomp['pupilId'].isin(pupils_toBeRemoved)]
df_subspt1 = df_subspt[~df_subspt['pupilId'].isin(pupils_toBeRemoved)]

df_subspt1 = utility.df.compute_customer_month(df_subspt1, cfg)

Complete loading data for subscription and lesson history!
By the cutoff date 2018-04-20, there are 1234 active subscriptions.
These subscribers shall be removed from the analysis because we have no evidence to know the lifetime of their subscriptions. 

In the first month of dataset starting from 2014-01-01, there are 154 renewal or new subscriptions.
These subscribers shall be removed from the analysis because we have no evidence to show if they renewed or newly joined. 

In summary, there are 1380/5685 subscribers being removed from the dataset in the analysis. 

Calculate customer month in the subscription table.


100%|██████████| 4305/4305 [00:05<00:00, 834.90it/s] 


In [14]:
# Stack Depth: 'replay' has stackDepth = 0
df_lesson_sd = df_lesson1[df_lesson1['lesson_type']!='replay']
sum_stackDepth = df_lesson_sd.groupby(['pupilId', 'date'])['stackDepth'].sum()
sum_stackDepth.rename('sum_stackDepth', inplace=True)
count_stackDepth = df_lesson_sd.groupby(['pupilId', 'date'])['stackDepth'].count()
count_stackDepth.rename('count_stackDepth', inplace=True)

# Help: 'tutor_pb' has total_help=0
df_lesson_hp = df_lesson1[df_lesson1['lesson_type']!='tutor_pb']
sum_help = df_lesson_hp.groupby(['pupilId', 'date'])['total_help'].sum()
sum_help.rename('sum_help', inplace=True)
count_help = df_lesson_hp.groupby(['pupilId', 'date'])['stackDepth'].count()
count_help.rename('count_help', inplace=True)

df = pd.concat([sum_stackDepth, count_stackDepth, sum_help, count_help], axis=1)
df.fillna(0, inplace=True)

sum_stackDepth  count_stackDepth  sum_help  count_help
pupilId date                                                              
655886  2014-04-18             2.0               2.0       0.0         1.0
        2014-04-19             1.0               1.0       4.0         1.0
        2014-04-20             2.0               2.0       6.0         1.0
        2014-04-21             3.0               3.0       6.0         2.0
        2014-05-18             5.0               5.0      18.0         4.0
        2014-10-21             9.0               9.0      17.0         5.0
        2014-10-22             3.0               3.0       3.0         2.0
        2014-12-23             9.0               9.0       5.0         5.0
        2015-03-06             3.0               3.0       1.0         2.0
        2015-05-17             7.0               7.0       1.0         2.0
658323  2013-12-12             1.0               1.0       1.0         1.0
        2013-12-27             2.0               2.0       0.0         1.0
        2014-02-23             1.0               1.0       0.0         0.0
        2016-03-10             9.0               9.0       7.0        11.0
        2016-03-11             1.0               1.0       0.0         1.0
        2016-03-13            27.0              27.0       4.0        14.0
        2016-03-14             2.0               2.0       0.0         1.0
        2016-03-16             5.0               5.0       1.0         2.0
        2016-03-17             5.0               5.0       0.0         3.0
        2016-03-19             5.0               5.0       0.0         3.0
        2016-03-21             6.0               6.0       1.0         3.0
        2016-03-22            11.0              11.0       4.0         5.0
        2016-03-23             5.0               5.0       0.0         6.0
        2016-03-25             0.0               0.0       3.0         4.0
        2016-03-29            12.0              12.0       2.0         6.0
        2016-04-12             4.0               4.0       3.0         9.0
        2016-04-13            14.0              14.0      11.0         8.0
        2016-04-16             5.0               5.0       0.0         5.0
        2016-04-19             6.0               6.0       0.0         3.0
        2016-04-20             6.0               6.0       3.0         3.0
...                            ...               ...       ...         ...
1076541 2016-09-18             3.0               3.0       0.0         1.0
        2016-09-20             2.0               2.0       0.0         1.0
        2016-09-25             4.0               4.0       0.0         2.0
        2016-10-02             6.0               6.0       0.0         3.0
        2016-10-04             4.0               4.0       0.0         2.0
        2016-10-15             6.0               6.0       7.0         3.0
        2016-10-19             2.0               2.0       0.0         1.0
        2016-10-22             5.0               5.0       0.0         3.0
        2016-10-23             6.0               6.0       0.0         3.0
        2016-11-05             8.0               8.0       0.0         4.0
        2016-11-06             4.0               4.0       0.0         2.0
        2016-11-10             2.0               2.0       0.0         1.0
        2016-11-13             4.0               4.0       0.0         2.0
        2016-11-15             2.0               2.0       0.0         1.0
        2016-11-19             6.0               6.0       0.0         3.0
        2016-11-22             3.0               3.0       0.0         2.0
        2016-12-18             5.0               5.0       1.0         2.0
        2016-12-20             2.0               2.0       0.0         1.0
        2016-12-21             2.0               2.0       0.0         1.0
        2016-12-22             2.0               2.0       0.0         1.0
        2016-12-23             2.0           